In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spooky-authors-csv/sample_submission.csv
/kaggle/input/spooky-authors-csv/train.csv
/kaggle/input/spooky-authors-csv/test.csv


In [2]:
import pandas as pd
import numpy as np

#tqdm for progress bars
from tqdm import tqdm

import xgboost as xgb
from sklearn.svm import SVC
from sklearn import preprocessing, decomposition, model_selection,metrics,pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

#keras/TF library
from keras.models import Sequential
from keras.layers.recurrent import LSTM,GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from tensorflow.keras.layers import BatchNormalization
from keras.utils import np_utils
from keras.layers import GlobalMaxPooling1D,Conv1D,MaxPooling1D,Flatten,Bidirectional,SpatialDropout1D,Embedding
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping

#nltk library
from nltk import word_tokenize
from nltk.corpus import stopwords

#ignore the warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
train  = pd.read_csv("../input/spooky-authors-csv/train.csv")
test   = pd.read_csv("../input/spooky-authors-csv/test.csv")
sample = pd.read_csv("../input/spooky-authors-csv/sample_submission.csv")

In [4]:
print("shape of training data:", train.shape)
print("shape of testing data:", test.shape)

shape of training data: (19579, 3)
shape of testing data: (8392, 2)


In [5]:
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [6]:
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [7]:
sample.head()

,id,EAP,HPL,MWS
0,id02310,0.403494,0.287808,0.308698
1,id24541,0.403494,0.287808,0.308698
2,id00134,0.403494,0.287808,0.308698
3,id27757,0.403494,0.287808,0.308698
4,id04081,0.403494,0.287808,0.308698


Note: There are three classes (authors). This is text classification problem into three classes. Kaggle mentions that the submissions are evaluated using multi-class logarithmic loss. Therefore, for each id, we need to predict the probability for each authors. 

labelencode the author column using LabelEncoder from scikit-learn

In [8]:
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(train["author"].values)
print(y[0:5])

[0 1 0 2 1]


train test split of the data using train_test_split function of scikit which has parameter test_size which decides fraction of the values to use as test data

In [9]:
# we will use 10% of data for testing
X_train, X_test, y_train, y_test = train_test_split(train.text.values,y,random_state=42,
                                                    test_size=0.1,shuffle=True)

In [10]:
# we can also pass countvectorizer parameters in TfidVectorizer
tfv = TfidfVectorizer(min_df=3,max_features=None,strip_accents='unicode',analyzer='word',token_pattern=r'\w{1,}',
                      ngram_range=(1,3),use_idf=1,smooth_idf=1,stop_words='english')

# max_features confines maximum number of words 

tfv.fit(list(X_train) + list(X_test))
X_train_tfv = tfv.transform(X_train)
X_test_tfv = tfv.transform(X_test)

In [11]:
print(X_train_tfv)

  (0, 14761)	0.3181283797747183
  (0, 13357)	0.4184575452230181
  (0, 12287)	0.43064224755100244
  (0, 7105)	0.42301057881313125
  (0, 6688)	0.44635106099095123
  (0, 4767)	0.40008455608345017
  (1, 14887)	0.39153056520357
  (1, 12875)	0.3449083818495815
  (1, 12809)	0.3084077740731038
  (1, 12324)	0.3554126762527559
  (1, 9703)	0.27395600238767365
  (1, 6671)	0.3410370927625761
  (1, 4688)	0.26215870355428317
  (1, 4606)	0.3554126762527559
  (1, 4106)	0.34627294568997125
  (2, 11244)	0.2572905992436592
  (2, 11046)	0.3148477991542896
  (2, 10869)	0.2910239963220062
  (2, 9873)	0.34888437299479774
  (2, 9850)	0.25638784821196636
  (2, 8179)	0.28700357069998056
  (2, 6390)	0.3373279740477724
  (2, 5009)	0.34888437299479774
  (2, 4520)	0.38146003200936524
  (2, 2367)	0.3148477991542896
  :	:
  (17618, 14988)	0.45135219095523216
  (17618, 10686)	0.37121890168782445
  (17618, 8099)	0.3324569127847727
  (17618, 7961)	0.37121890168782445
  (17618, 6016)	0.48027623090755894
  (17618, 2609)	0.

In [12]:
def multiclass_logloss(actual,predicted,eps=1e-15):
    
    #converting the 'actual' values to binary values if it's 
    #not binary values
    
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0],predicted.shape[1]))
        
        for i, val in enumerate(actual):
            actual2[i,val] = 1
        actual = actual2
    
    #clip function truncates the number between
    #a max number and min number
    clip = np.clip(predicted,eps,1-eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0/ rows * vsota 

In [13]:
# Fitting Logistic Regression on TFIDF
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=1.0)
clf.fit(X_train_tfv,y_train)
prediction = clf.predict_proba(X_test_tfv)

print("logloss: %0.3f" % multiclass_logloss(y_test,prediction))

logloss: 0.583


In [14]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

ctv.fit(list(X_train)+list(X_test))
X_train_ctv = ctv.transform(X_train)
X_test_ctv = ctv.transform(X_test)

In [15]:
clf = LogisticRegression(C=1.0)
clf.fit(X_train_ctv,y_train)
prediction = clf.predict_proba(X_test_ctv)

print("logloss: %0.3f" % multiclass_logloss(y_test,prediction))

logloss: 0.537


# Navie Bayes

In [16]:
# tfidf
clf = MultinomialNB()
clf.fit(X_train_tfv,y_train)

prediction = clf.predict_proba(X_test_tfv)

print("logloss: %0.3f" % multiclass_logloss(y_test,prediction))

logloss: 0.582


In [17]:
# counts
clf = MultinomialNB()
clf.fit(X_train_ctv,y_train)

prediction = clf.predict_proba(X_test_ctv)

print("logloss: %0.3f" % multiclass_logloss(y_test,prediction))

logloss: 0.502


# SVM

SVM is very slow algorithm so it takes lot of time to fit so we will
use Singular Value Decomposition before applying SVM
and we will also standardize the data.

In [18]:
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(X_train_tfv)
X_train_svd = svd.transform(X_train_tfv)
X_test_svd = svd.transform(X_test_tfv)

scl = preprocessing.StandardScaler()
scl.fit(X_train_svd)

X_train_svd_scl = scl.transform(X_train_svd)
X_test_svd_scl = scl.transform(X_test_svd)

In [19]:
svm = SVC(C=1.0,probability=True)

svm.fit(X_train_svd_scl,y_train)
prediction = svm.predict_proba(X_test_svd_scl)

print("logloss: %0.3f" % multiclass_logloss(y_test,prediction))

logloss: 0.712


# Xgboost

In [20]:
# on TF-IDF
clf = xgb.XGBClassifier(max_depth=7,n_estimators=200,colsample_bytree=0.8,subsample=0.8,nthread=10,learning_rate=0.1)

clf.fit(X_train_tfv.tocsc(),y_train)
prediction = clf.predict_proba(X_test_tfv.tocsc())

print("logloss: %0.3f" % multiclass_logloss(y_test,prediction))

logloss: 0.778


In [21]:
# on SVD
clf = xgb.XGBClassifier(max_depth=7,n_estimators=200,colsample_bytree=0.8,subsample=0.8,nthread=10,learning_rate=0.1)

clf.fit(X_train_svd,y_train)
prediction = clf.predict_proba(X_test_svd)

print("logloss: %0.3f" % multiclass_logloss(y_test,prediction))

logloss: 0.722


# Hyperparameter Optimization

In [22]:
mll_scorer = metrics.make_scorer(multiclass_logloss, greater_is_better=False,needs_proba=True)

svd = decomposition.TruncatedSVD()
scl = preprocessing.StandardScaler()
lr_model = LogisticRegression()

clf = pipeline.Pipeline([('svd',svd),
                         ('scl',scl),
                         ('lr',lr_model)])

In [23]:
params_grid = {'svd__n_components':[120,180],
               'lr__C':[0.1,1.0,10],
               'lr__penalty':['l1','l2']}

In [24]:
model = GridSearchCV(estimator=clf,param_grid=params_grid,scoring=mll_scorer,
                     verbose=10,n_jobs=-1,refit=True,cv=2)

#fitting the model
model.fit(X_train_tfv,y_train)

print('Best score: %0.3f' % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(params_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 12 candidates, totalling 24 fits
Best score: -0.736
Best parameters set:
	lr__C: 0.1
	lr__penalty: 'l2'
	svd__n_components: 180


In [25]:
# Grid search for NB
nb = MultinomialNB()

clf = pipeline.Pipeline([('nb',nb)])

params_grid = {'nb__alpha':[0.001,0.01,0.1,1,10,100]}

model  = GridSearchCV(estimator=clf,param_grid=params_grid,scoring=mll_scorer,verbose=10,n_jobs=-1,refit=True,cv=2)

model.fit(X_train_tfv,y_train)

print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(params_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
Best score: -0.491
Best parameters set:
	nb__alpha: 0.1


# Word2Vec

In [26]:
#gensim library allow us to access pre trained embeddings
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

In [27]:
#download the dataset and return it as object
model_twitter_glove = api.load("glove-twitter-25") #here 25 is dimenssion of the data

[==================================================] 100.0% 104.8/104.8MB downloaded


In [28]:
#we need to word, vec dictionary before fitting it to models
embedding_index = {}

all_words = list(model_twitter_glove.key_to_index.keys())
#words in gensim model is stored as "key":vector object pair

for word in all_words:
    embedding_index[word] = model_twitter_glove.get_vector(word)

print('Total words in embeddings %d' % len(embedding_index))

Total words in embeddings 1193514


In [29]:
#getting stop words from nltk library
stop_words = stopwords.words('english')

def sen2vec(s):
    # lowe the letters, tokenize them , remove stop_words, remove numbers
    words = str(s).lower()
    words = word_tokenize(s)
    words = [w for w in words if w not in stop_words]
    words = [w for w in words if w.isalpha()]
    
    M = []
    for w in words:
        #try because word might not present in index.
        try:
            M.append(embedding_index[w])
        except:
            continue
    
    M = np.array(M)
    v = M.sum(axis=0)
    
    if type(v) != np.ndarray:
        #25 because that is dimension of out word embedding
        return np.zeros(25)
    
    return v/np.sqrt((v** 2).sum())

In [30]:
#converting every sentence to word embedding
X_train_glove = [sen2vec(s) for s in tqdm(X_train)]
X_test_glove = [sen2vec(s) for s in tqdm(X_test)]

100%|██████████| 1958/1958 [00:00<00:00, 2784.27it/s]


In [31]:
X_train_glove = np.array(X_train_glove)
X_test_glove = np.array(X_test_glove)

# XGBoost on GloVe

In [32]:
clf = xgb.XGBClassifier(n_estimators=200,nthread=10,silent=False)
clf.fit(X_train_glove, y_train)

predictions = clf.predict_proba(X_test_glove)

print ("logloss: %0.3f " % multiclass_logloss(y_test, predictions))

[13:52:41] WARNING: ../src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


logloss: 0.984 


# Simple NN

In [33]:
# Remember to scale the data before feeding to the NN
scl = preprocessing.StandardScaler()

X_train_glove_scl = scl.fit_transform(X_train_glove)
X_test_glove_scl = scl.transform(X_test_glove)

y_train_enc = np_utils.to_categorical(y_train)
y_test_enc = np_utils.to_categorical(y_test)

In [34]:
model = Sequential()

model.add(Dense(25,input_dim=25,activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(25,input_dim=25,activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam')

2022-08-31 13:53:24.685850: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [35]:
# let's train just for 5 epochs
model.fit(X_train_glove_scl,y=y_train_enc,batch_size=50,epochs=5,verbose=1,
          validation_data=(X_test_glove_scl,y_test_enc))

2022-08-31 13:53:26.688031: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
353/353 [==============================] - 2s 3ms/step - loss: 1.2179 - val_loss: 1.0028
Epoch 2/5
353/353 [==============================] - 1s 2ms/step - loss: 1.0474 - val_loss: 0.9739
Epoch 3/5
353/353 [==============================] - 1s 3ms/step - loss: 1.0081 - val_loss: 0.9576
Epoch 4/5
353/353 [==============================] - 1s 3ms/step - loss: 0.9910 - val_loss: 0.9472
Epoch 5/5
353/353 [==============================] - 1s 3ms/step - loss: 0.9784 - val_loss: 0.9425


# Simple RNN

In [36]:
# remember to totenize the words before feeding to RNN
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(X_train)+list(X_test))
X_train_sec = token.texts_to_sequences(X_train)
X_test_sec = token.texts_to_sequences(X_test)

X_train_pad = sequence.pad_sequences(X_train_sec,maxlen=max_len)
X_test_pad = sequence.pad_sequences(X_test_sec,maxlen=max_len)

word_index = token.word_index

In [37]:
model = Sequential()
# we are not using pretrainde embedding yet.
model.add(Embedding(len(word_index)+1,25,input_length=max_len))
model.add(SimpleRNN(100))
model.add(Dense(3))
model.compile(loss='categorical_crossentropy',optimizer='adam')


In [38]:
model = Sequential()
# we are not using pretrainde embedding yet.
model.add(Embedding(len(word_index)+1,25,input_length=max_len))
model.add(SimpleRNN(100))
model.add(Dense(3))
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [39]:
model.fit(X_train_pad,y=y_train_enc,epochs=5,batch_size=100,validation_data=(X_test_pad,y_test_enc))

Epoch 1/5
177/177 [==============================] - 12s 62ms/step - loss: 10.3232 - val_loss: 10.7097
Epoch 2/5
177/177 [==============================] - 11s 64ms/step - loss: 11.1906 - val_loss: 10.7097
Epoch 3/5
177/177 [==============================] - 11s 61ms/step - loss: 11.1906 - val_loss: 10.7097
Epoch 4/5
177/177 [==============================] - 11s 60ms/step - loss: 11.1906 - val_loss: 10.7097
Epoch 5/5
177/177 [==============================] - 11s 64ms/step - loss: 11.1906 - val_loss: 10.7097


In [40]:
embedding_matrix = np.zeros((len(word_index)+1,25)) #25 because we have word vector of dim 25

for word,i in word_index.items():
    #we use get() so it returns None if word is not found
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [41]:
model = Sequential()
# we are not using pretrainde embedding yet.
model.add(Embedding(len(word_index)+1,25,
                    weights=[embedding_matrix],
                    input_length=max_len,
                    trainable=False))
#as the weight are predefined trainable is False
model.add(SimpleRNN(100))
model.add(Dropout(0.2))

model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [42]:
model.fit(X_train_pad,y=y_train_enc,epochs=5,batch_size=100,validation_data=(X_test_pad,y_test_enc))

Epoch 1/5
177/177 [==============================] - 10s 51ms/step - loss: 1.0695 - val_loss: 0.9783
Epoch 2/5
177/177 [==============================] - 9s 51ms/step - loss: 0.9691 - val_loss: 0.9071
Epoch 3/5
177/177 [==============================] - 9s 53ms/step - loss: 0.9270 - val_loss: 0.8791
Epoch 4/5
177/177 [==============================] - 9s 49ms/step - loss: 0.9082 - val_loss: 0.8840
Epoch 5/5
177/177 [==============================] - 9s 48ms/step - loss: 0.8985 - val_loss: 0.9037


# LSTM

In [43]:
model = Sequential()

model.add(Embedding(len(word_index)+1,25,
                    weights=[embedding_matrix],
                    input_length=max_len,
                    trainable=False))

model.add(SpatialDropout1D(0.3))
model.add(LSTM(100,dropout=0.3,recurrent_dropout=0.3))

model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [44]:
model.fit(X_train_pad,y=y_train_enc,batch_size=100,epochs=10,verbose=1,
          validation_data=(X_test_pad,y_test_enc))

Epoch 1/10
150/177 [========================>.....] - ETA: 9s - loss: 1.0857[CV 2/2; 1/12] START lr__C=0.1, lr__penalty=l1, svd__n_components=120...........
[CV 2/2; 1/12] END lr__C=0.1, lr__penalty=l1, svd__n_components=120;, score=nan total time=   1.9s
[CV 2/2; 3/12] START lr__C=0.1, lr__penalty=l2, svd__n_components=120...........
[CV 2/2; 3/12] END lr__C=0.1, lr__penalty=l2, svd__n_components=120;, score=-0.776 total time=   2.3s
[CV 2/2; 5/12] START lr__C=1.0, lr__penalty=l1, svd__n_components=120...........
[CV 2/2; 5/12] END lr__C=1.0, lr__penalty=l1, svd__n_components=120;, score=nan total time=   2.0s
[CV 2/2; 6/12] START lr__C=1.0, lr__penalty=l1, svd__n_components=180...........
[CV 2/2; 6/12] END lr__C=1.0, lr__penalty=l1, svd__n_components=180;, score=nan total time=   2.9s
[CV 2/2; 8/12] START lr__C=1.0, lr__penalty=l2, svd__n_components=180...........
[CV 2/2; 8/12] END lr__C=1.0, lr__penalty=l2, svd__n_components=180;, score=-0.746 total time=   3.9s
[CV 2/2; 11/12] ST

# GRU

In [45]:
model = Sequential()

model.add(Embedding(len(word_index)+1,25,
                    weights=[embedding_matrix],
                    input_length=max_len,
                    trainable=False))

model.add(SpatialDropout1D(0.3))
model.add(GRU(100))

model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [46]:
model.fit(X_train_pad,y=y_train_enc,batch_size=100,epochs=5,verbose=1,
          validation_data=(X_test_pad,y_test_enc))

Epoch 1/5
177/177 [==============================] - 45s 239ms/step - loss: 1.0936 - val_loss: 1.0260
Epoch 2/5
177/177 [==============================] - 42s 239ms/step - loss: 1.0272 - val_loss: 0.9429
Epoch 3/5
177/177 [==============================] - 42s 240ms/step - loss: 0.9769 - val_loss: 0.9044
Epoch 4/5
177/177 [==============================] - 41s 233ms/step - loss: 0.9528 - val_loss: 0.8953
Epoch 5/5
177/177 [==============================] - 42s 236ms/step - loss: 0.9322 - val_loss: 0.8818


# Bidirectional LSTM

In [47]:
model = Sequential()

model.add(Embedding(len(word_index)+1,25,
                    weights=[embedding_matrix],
                    input_length=max_len,
                    trainable=False))

model.add(Bidirectional(LSTM(25, dropout=0.3, recurrent_dropout=0.3)))
    
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [48]:
model.fit(X_train_pad,y=y_train_enc,batch_size=100,epochs=5,verbose=1,
          validation_data=(X_test_pad,y_test_enc))

Epoch 1/5
177/177 [==============================] - 60s 289ms/step - loss: 1.0481 - val_loss: 0.9628
Epoch 2/5
177/177 [==============================] - 49s 275ms/step - loss: 0.9857 - val_loss: 0.9126
Epoch 3/5
177/177 [==============================] - 48s 274ms/step - loss: 0.9539 - val_loss: 0.8640
Epoch 4/5
177/177 [==============================] - 49s 276ms/step - loss: 0.9311 - val_loss: 0.8489
Epoch 5/5
177/177 [==============================] - 49s 275ms/step - loss: 0.9149 - val_loss: 0.8333
